In [1]:
import numpy as np

import astropy.units as u
from astroquery.gaia import Gaia
from astropy.table import Table, Column
from astropy.io import ascii
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

In [2]:
table = Gaia.load_table('gaiaedr3.gaia_source')

Retrieving table 'gaiaedr3.gaia_source'


In [3]:
SPEC_CAT = ascii.read("../reference_data/color_table.txt").to_pandas()

def gaia_sptype(color, cat=SPEC_CAT, color_name='G-Rp'):
    diff = cat[color_name] - color
    absdiff = abs(diff)
    sptype = cat.loc[absdiff.idxmin()]['SpT'].replace('V', '')
    return sptype

In [4]:
for col in table.columns:
    print(col.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [23]:
#def gaia2mmt(nstars, )
job = Gaia.launch_job_async(
    "select top 10000 designation,ref_epoch,ra,dec,pmra,pmdec,phot_g_mean_mag as gmag,g_rp "
    "from gaiaedr3.gaia_source where (phot_g_mean_mag > 8. AND dec > -25 AND pmra IS NOT NULL) order by gmag"
)

INFO: Query finished. [astroquery.utils.tap.core]


In [24]:
r = job.get_results()
r['sptype'] = [gaia_sptype(g_rp) for g_rp in r['g_rp']]
ids = [f"GEDR3_{s[-10:]}" for s in r['designation']]

In [25]:
t_obs = Time(r['ref_epoch'].data, format='decimalyear')
t_ref = Time('J2000')
c = SkyCoord(
    ra=r['ra'],
    dec=r['dec'],
    #distance=Distance(100 * u.kpc),  # this is kind of a hack to work around astropy pedantry
    pm_ra_cosdec=r['pmra'],
    pm_dec=r['pmdec'],
    obstime=t_obs,
    frame='icrs'
)
c_j2000 = c.apply_space_motion(t_ref)
ra_2000 = c_j2000.ra.to_string(unit=u.hr, sep=":", precision=4, pad=True)
dec_2000 = c_j2000.dec.to_string(unit=u.deg, sep=":", alwayssign=True, precision=3, pad=True)
pmra_mmt = r['pmra'].data / 15. / 10. / np.cos(c_j2000.dec)  # MMT uses seconds of time per century
pmdec_mmt = r['pmdec'].data / 10.  # MMT uses seconds of arc per century


/home/tim/conda/envs/py39/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 9999 of "distance overridden (Note 6)", 1 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [26]:
out_tab = Table()
out_tab['ID'] = Column(ids, format='<15')
out_tab['RA'] = Column(ra_2000)
out_tab['Dec'] = Column(dec_2000)
out_tab['PM_RA'] = Column(pmra_mmt.data, format=".4f")
out_tab['PM_Dec'] = Column(pmdec_mmt.data, format=".4f")
out_tab['Mag'] = Column(r['gmag'].data, format=".2f")
out_tab['Sp_Type'] = Column(r['sptype'])
out_tab.add_column('J2000.0', name='Epoch')
#out_tab = out_tab[not np.isnan(out_tab['PM_RA'])]
out_tab.write("test.cat", format='ascii.fixed_width_no_header', delimiter=None, overwrite=True, bookend=False)

In [11]:
r

designation,ref_epoch,ra,dec,pmra,pmdec,gmag,g_rp,sptype
,yr,deg,deg,mas / yr,mas / yr,mag,mag,
object,float64,float64,float64,float64,float64,float32,float32,str4
Gaia EDR3 4382944375300246528,2016.0,250.71768526703008,0.07525482803124896,11.769904520217906,3.683946654793089,8.000009,0.7918658,K6
Gaia EDR3 5616982973026239744,2016.0,111.24600126675567,-24.426311998237164,-8.501098184244631,5.64171298412784,8.000086,-0.12911797,B9
Gaia EDR3 3945629435127382528,2016.0,185.74724024916904,16.243332374121266,19.41486579332065,-22.33261375460637,8.000116,0.6855931,K3.5
Gaia EDR3 2036262278731833344,2016.0,287.9051748825728,26.397090985854057,-13.595149561492466,-21.87414342033881,8.000144,0.8176441,K6.5
Gaia EDR3 3620575830445064704,2016.0,208.54210924154953,-5.773213262372844,-24.048271047237524,0.15523560215845558,8.000181,0.8807292,K8
Gaia EDR3 976527442574792192,2016.0,109.66576343874337,48.12328457227269,6.017020820820831,9.127265484636935,8.000214,0.72763634,K4.5
Gaia EDR3 359589818965062272,2016.0,29.207144841444887,51.417521661928085,8.854878661123966,-23.270741330090175,8.00022,0.87609005,K8
Gaia EDR3 451958622584743424,2016.0,36.71957573728719,52.11740588931467,-1.757130434088841,-1.8186743882921248,8.000224,1.2220955,M4


In [16]:
r.filled().write("gaia.cat", format='ascii.fixed_width_no_header', delimiter=None, overwrite=True, bookend=False)

In [91]:
r['pmra'].data

masked_array(data=[14.276452511494501, 8.592570027988241,
                   -56.70330482029109, ..., -16.28740924268928,
                   2.6886430063114286, -1.087576964993453],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20)